In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import cv2

In [2]:
# ResNet
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def conv_block(input_tensor, filters, kernel_size, strides=(2, 2), padding="same"):
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def identity_block(input_tensor, filters, kernel_size):
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")(x)
    x = BatchNormalization()(x)
    
    x = Add()([x, input_tensor])
    x = Activation("relu")(x)
    return x

def ResNet50(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)
    
    x = conv_block(input_tensor, filters=64, kernel_size=(7, 7), strides=(2, 2))
    x = MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)

    # Stage 2
    x = identity_block(x, filters=64, kernel_size=(3, 3))
    x = identity_block(x, filters=64, kernel_size=(3, 3))
    x = identity_block(x, filters=64, kernel_size=(3, 3))

    # Stage 3
    x = conv_block(x, filters=128, kernel_size=(3, 3), strides=(2, 2))
    x = identity_block(x, filters=128, kernel_size=(3, 3))
    x = identity_block(x, filters=128, kernel_size=(3, 3))
    x = identity_block(x, filters=128, kernel_size=(3, 3))

    # Stage 4
    x = conv_block(x, filters=256, kernel_size=(3, 3), strides=(2, 2))
    x = identity_block(x, filters=256, kernel_size=(3, 3))
    x = identity_block(x, filters=256, kernel_size=(3, 3))
    x = identity_block(x, filters=256, kernel_size=(3, 3))
    x = identity_block(x, filters=256, kernel_size=(3, 3))
    x = identity_block(x, filters=256, kernel_size=(3, 3))

    # Stage 5
    x = conv_block(x, filters=512, kernel_size=(3, 3), strides=(2, 2))
    x = identity_block(x, filters=512, kernel_size=(3, 3))
    x = identity_block(x, filters=512, kernel_size=(3, 3))
    
    x = AveragePooling2D((7, 7))(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation="softmax")(x)
    
    model = Model(inputs=input_tensor, outputs=x, name="ResNet50")
    return model

# Example usage:
input_shape = (224, 224, 3)  # Input shape of images
num_classes = 2  # Number of classes (binary classification)

model = ResNet50(input_shape, num_classes)
model.summary()


Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │     36,928 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     36,928 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 56, 56,    │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │     36,928 │ activation_2[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     36,928 │ activation_3[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_4[0][0]  

 Total params: 18,033,666 (68.79 MB)

 Trainable params: 18,020,226 (68.74 MB)

 Non-trainable params: 13,440 (52.50 KB)

In [3]:
import tensorflow as tf

# Allow gpu usage
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
try:
    tf.config.experimental.set_memory_growth = True
except Exception as ex:
    print(ex)

[]


In [4]:
def load_images_from_folder(folder_path, img_size=(224, 224)):
    images = []
    labels = []

    for filename in os.listdir(folder_path):
        label = filename  # Assuming file name format is label.image_number.jpg
        if label == "Cancer":
            label1 = 1 # Convert label to integer if necessary
        else:
            label1 = 0
        
        for photoname in os.listdir(folder_path + "/" + label):
            img = cv2.imread(os.path.join(folder_path, filename, photoname))
            img = cv2.resize(img, img_size)  # Resize image to desired dimensions
            images.append(img)
            labels.append(label1)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [5]:
imagesTrain, labelsTrain = load_images_from_folder("Train1")


In [6]:
imagesTest, labelsTest = load_images_from_folder("Test1")


In [7]:
imagesTrain, labelsTrain = load_images_from_folder("Train1")
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(imagesTrain, labelsTrain, epochs=10, validation_split=0.2)

Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 190s 3s/step - accuracy: 0.5262 - loss: 1.6183 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 176s 3s/step - accuracy: 0.6075 - loss: 0.6845 - val_accuracy: 0.9040 - val_loss: 0.1439
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 178s 3s/step - accuracy: 0.6348 - loss: 0.6631 - val_accuracy: 0.3862 - val_loss: 0.7873
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 172s 3s/step - accuracy: 0.5969 - loss: 0.6806 - val_accuracy: 0.5357 - val_loss: 0.6525
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 173s 3s/step - accuracy: 0.6291 - loss: 0.6639 - val_accuracy: 0.3214 - val_loss: 0.9299
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 187s 3s/step - accuracy: 0.6519 - loss: 0.6316 - val_accuracy: 0.1272 - val_loss: 1.0814
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.6362 - loss: 0.6520 - val_accuracy: 0.0737 - val_loss: 1.2184
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 188s 3s/step - accuracy: 0.6230 - loss: 0.6655 - val_accuracy: 0.1585

In [8]:
test_loss, test_accuracy = model.evaluate(imagesTest, labelsTest)
print(f"Test Accuracy: {test_accuracy}")
test_loss1, test_accuracy1 = model.evaluate(imagesTrain, labelsTrain)
print(test_accuracy1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 792ms/step - accuracy: 0.4935 - loss: 1.9721
Test Accuracy: 0.5593220591545105
70/70 ━━━━━━━━━━━━━━━━━━━━ 61s 876ms/step - accuracy: 0.4696 - loss: 2.5441
0.6074140071868896
